In [1]:
!cat cpu-limits.yaml

apiVersion: v1
kind: ResourceQuota
metadata:
  name: compute-resources
spec:
  hard:
    limits.cpu: "400m"

In [2]:
!cat tomcat-deployments.yaml

apiVersion: apps/v1beta2
kind: Deployment
metadata:
  name: tomcat-deployment
spec:
  selector:
    matchLabels:
      app: tomcat
  replicas: 3
  template:
    metadata:
      labels:
        app: tomcat
    spec:
      containers:
      - name: tomcat
        image: tomcat:9.0
        ports:
        - containerPort: 8080
        resources:
          limits:
            cpu: "200m"

In [3]:
!kubectl create namespace cpu-limited-tomcat

namespace/cpu-limited-tomcat created


In [4]:
%%bash
kubectl create -f ./cpu-limits.yaml --namespace=cpu-limited-tomcat

resourcequota/compute-resources created


In [5]:
!kubectl get resourcequota --namespace=cpu-limited-tomcat

NAME                CREATED AT
compute-resources   2018-09-12T14:47:12Z


In [6]:
!kubectl apply -f ./tomcat-deployments.yaml --namespace=cpu-limited-tomcat

deployment.apps/tomcat-deployment created


In [9]:
!kubectl describe deployment tomcat-deployment \
--namespace=cpu-limited-tomcat

Name:                   tomcat-deployment
Namespace:              cpu-limited-tomcat
CreationTimestamp:      Wed, 12 Sep 2018 16:49:09 +0200
Labels:                 <none>
Annotations:            deployment.kubernetes.io/revision=1
                        kubectl.kubernetes.io/last-applied-configuration={"apiVersion":"apps/v1beta2","kind":"Deployment","metadata":{"annotations":{},"name":"tomcat-deployment","namespace":"cpu-limited-tomcat"},"spec":{"rep...
Selector:               app=tomcat
Replicas:               3 desired | 2 updated | 2 total | 2 available | 1 unavailable
StrategyType:           RollingUpdate
MinReadySeconds:        0
RollingUpdateStrategy:  25% max unavailable, 25% max surge
Pod Template:
  Labels:  app=tomcat
  Containers:
   tomcat:
    Image:      tomcat:9.0
    Port:       8080/TCP
    Host Port:  0/TCP
    Limits:
      cpu:        200m
    Environment:  <none>
    Mounts:       <none>
  Volumes:        <none>
Conditions:
  Type             Status  Reason
  ---

<font color=red> 400 x 3 = 1200%. Third replica cannot be created because it exceeds the capacity. So kubernetes fails, without creating 2 replicas. </font>

In [15]:
!!kubectl describe deployment tomcat-deployment \
--namespace=cpu-limited-tomcat | sed -n '/Conditions/,/EndPattern/p'

['Conditions:',
 '  Type             Status  Reason',
 '  ----             ------  ------',
 '  Available        False   MinimumReplicasUnavailable',
 '  ReplicaFailure   True    FailedCreate',
 '  Progressing      False   ProgressDeadlineExceeded',
 'OldReplicaSets:    <none>',
 'NewReplicaSet:     tomcat-deployment-7bc8d779d4 (2/3 replicas created)',
 'Events:',
 '  Type    Reason             Age   From                   Message',
 '  ----    ------             ----  ----                   -------',
 '  Normal  ScalingReplicaSet  14m   deployment-controller  Scaled up replica set tomcat-deployment-7bc8d779d4 to 3']

<font color=red>Conditions give information on the conditions for creating the resource. Events gives information on different events. Combining both information makes the trobleshooting part easier.</font>

#### Commands For Cut & Paste

    kubectl create namespace <namespace name>
    kubectl get namespace
    kubectl create namespace cpu-limited-tomcat
    kubectl create -f ./cpu-limits.yaml —namespace=cpu-limited-tomcat (from the GitHub repo directory for this lecture)
    kubectl apply -f ./tomcat-deployment.yaml —namespace=cpu-limited-tomcat (from the GitHub repo directory for this lecture)
    kubectl describe deployment tomcat-deployment —namespace=cpu-limited-tomcat